# HATS Translation of: Investigating Variable Stars in DP0.2
**Original Author:** Sabina Ustamujic  
**Translated to HATS by:** Olivia Lynn

- Last verified to run: 2023-04-14
- LSST Science Piplines version: Weekly 2023_07
- Container Size: medium

**Description:** Use catalog data to identify variable stars, plot their lightcurves and calculate their structure function.  
**LSST Data Products:** TAP tables dp02_dc2_catalogs.Object, MatchesTruth, TruthSummary, CcdVisit, DiaObject, DiaSource, ForcedSourceOnDiaObject  
**Packages:** numpy, matplotlib, lsst.rsp.get_tap_service, pandas, numba    
**Credit:** Developed by Sabina Ustamujic. Please consider acknowledging Sabina if this notebook is used for the preparation of journal articles, software releases, or other notebooks. This notebook is based in part on material originally developed by Jeff Carlin, Melissa Graham, Ryan Lau and the Rubin Community Engagement Team for Data Preview 0. Special thanks to Laura Venuti and Sara Bonito for the support in the analysis.  


## 1. Introduction

### Description

This notebook demonstrates the use of the TAP service to query the time-domain data products to select variable stars, get their light curves, and calculate their structure functions.

In Section 2 the data products of difference image analysis (DIA), in particular the lightcurve summary statistic parameters available in the `DiaObjects` table, are used to identify a sample of variable stars candidates from the DP0.2 data set and extract their lightcurves using the time-series photometry from `ForcedSourceOnDiaObject` table. Then the coordinates of the identified objects are used to find them in the `Object` table, and check if they are classified as variable stars in the `TruthSummary` table.

In Section 3 the extracted lightcurves for each object are used to calculate their structure functions (SFs). 
The method of SFs (Simonetti et al. 1985 ; Hughes et al. 1992 ; de Vries et al. 2003) was developed to probe the characteristic timescales of variability from lightcurves and it has been more recently implemented to study Young Stellar Objects (YSOs) variability by Sergison et al. (2020) and applied by Venuti et al. (2021).
The method consists of extracting every timescale of variability τ encompassed by the time series and, for each τ, computing the average amplitude of normalized flux variability among all pairs of points in the light curve that are spaced by that time interval τ. The SF is then defined as the average variability amplitude measured within the light curve as a function of τ.

Finally two examples of YSO lightcurves are imported and the SF analysis is performed as was done in the previous section.

### 1.1 Package imports

In [2]:
# NOTE: use data.lsdb.io/rubin
# as long as you're in USDF
# latest release is weird bc tables changed names - not called diaForcedSource anymore, something else

In [3]:
#LIV
import hats
import lsdb
from hats.io.validation import is_valid_catalog
from dask.distributed import Client

In [4]:
#LIV
client = Client(n_workers=4, memory_limit="auto")
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 4
Total threads: 28,Total memory: 405.17 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:45695,Workers: 4
Dashboard: http://127.0.0.1:8787/status,Total threads: 28
Started: Just now,Total memory: 405.17 GiB
Comm: tcp://127.0.0.1:39421,Total threads: 7
Dashboard: http://127.0.0.1:41831/status,Memory: 101.29 GiB
Nanny: tcp://127.0.0.1:40669,


In [6]:
# General
import numpy as np
import matplotlib.pyplot as plt
plt.style.use('tableau-colorblind10')

# LSST package for TAP queries
#from lsst.rsp import get_tap_service

# SF calculation
import pandas as pd
from numba import njit

### 1.2 Define functions and parameters

Set `matplotlib` to show plots inline, within the notebook.

In [7]:
%matplotlib inline

Set up colors and plot symbols corresponding to the ugrizy bands. These colors are the same as those used for ugrizy bands in Dark Energy Survey (DES) publications, and in most tutorial notebooks.

In [8]:
filters = ['u', 'g', 'r', 'i', 'z', 'y']
filters += ['V', 'G', 'R', 'I', 'B']   # added for V2492_Cyg (Section 4)

symbols = {'u': 'o', 'g': '^', 'r': 'v', 'i': 's', 'z': '*', 'y': 'p'}
symbols.update({'G':'.', 'R':'.', 'I':'.', 'V':'.', 'B':'.'})   # added for V2492_Cyg (Section 4)

colors = {'u': '#56b4e9', 'g': '#008060', 'r': '#ff4000',
          'i': '#850000', 'z': '#6600cc', 'y': '#000000'}
colors.update({'V': '#56b4e9', 'G': '#008060', 'R': '#ff4000', 'I': '#850000', 'B': '#000000'})   # added for V2492_Cyg (Section 4)

Start the TAP service, which we will use for all data retrieval in this notebook.

In [9]:
#service = get_tap_service()

## 2. Identify candidate variable stars from Catalogs

Data products from difference image analysis (DIA), where sources are detected and measured on the difference image resulting from subtracting an approriately scaled deep coadd template of the same sky area from a processed visit image (PVI), are a good place to check for variable objects (see 07b Tutorial Notebook).

The three DIA products used here are:

- `DiaObject`: table of all spatially unique objects detected in all difference images (all `DiaSources` associated by coordinate)
- `DiaSource`: table of sources detected in the difference images
- `ForcedSourceOnDiaObject`: table of forced photometry in all PVI and all difference images at the location of all `DiaObjects`

### 2.1 Query DiaObject and ForcedSourceOnDiaObject to Retrieve Candidates and Photometry

The DP0.2 `DiaObject` table contains summary statistics for each object, which can be used to identify candidate variables.

Here are the criteria we apply to our selection from `DiaObject`:

 1. `gTOTFluxSigma`/`gTOTFluxMean` > 0.15 -- the scatter in measured fluxes is larger than 15% relative to the mean
 2. `gPSFluxNdata` > 60 -- at least 60 observations in g band

The query below implements these criteria and performs a table JOIN between `DiaObject` and `ForcedSourceOnDiaObject` to extract the forced-photometry measurements, and between `ForcedSourceOnDiaObject` and `CcdVisit` tables in order to get the MJD of each visit. 

In [27]:
#LIV

rubin_data = lsdb.read_hats(
    '/sdf/data/rubin/shared/lsdb_commissioning/hats/w_2025_11/dia_object_lc', 
    margin_cache='/sdf/data/rubin/shared/lsdb_commissioning/hats/w_2025_11/dia_object_lc_5arcs',
    columns=["diaObjectId", "ra", "dec", "diaSource", "diaObjectForcedSource"]
    )

In [28]:
rubin_data

,diaObjectId,ra,dec,diaSource,diaObjectForcedSource
npartitions=237,,,,,
"Order: 0, Pixel: 0",int64[pyarrow],double[pyarrow],double[pyarrow],"struct<visit: list<element: int64>, detector: ...","struct<parentObjectId: list<element: int64>, c..."
"Order: 0, Pixel: 4",...,...,...,...,...
...,...,...,...,...,...
"Order: 6, Pixel: 35957",...,...,...,...,...
"Order: 3, Pixel: 562",...,...,...,...,...


In [29]:
rubin_data.columns

Index(['diaObjectId', 'ra', 'dec', 'diaSource', 'diaObjectForcedSource'], dtype='object')

In [30]:
rubin_data.hc_structure.schema

diaObjectId: int64
ra: double
dec: double
diaSource: struct<visit: list<element: int64>, detector: list<element: int16>, band: list<element: string>, ssObjectId: list<element: int64>, parentDiaSourceId: list<element: int64>, midpointMjdTai: list<element: double>, bboxSize: list<element: int64>, time_processed: list<element: timestamp[ns]>, ra: list<element: double>, dec: list<element: double>, raErr: list<element: float>, decErr: list<element: float>, ra_dec_Cov: list<element: float>, x: list<element: double>, y: list<element: double>, xErr: list<element: float>, yErr: list<element: float>, apFlux: list<element: double>, apFluxErr: list<element: double>, snr: list<element: double>, psfFlux: list<element: double>, psfFluxErr: list<element: double>, psfChi2: list<element: float>, psfNdata: list<element: int32>, trailFlux: list<element: double>, trailRa: list<element: double>, trailDec: list<element: double>, trailLength: list<element: double>, trailAngle: list<element: double>, dipoleMea

| Concept                 | TAP Reference                          | HATS Reference                          |
|-------------------------|----------------------------------------|-----------------------------------------|
| DIA Object ID           | `diao.diaObjectId as Object_ID`        | `diaObjectId` (top-level)               |
| Right Ascension (RA)    | `diao.ra as RA`                        | `ra` (top-level)                        |
| Declination (Dec)       | `diao.decl as DEC`                     | `dec` (top-level)                       |
| gPSFluxNdata            | `diao.gPSFluxNdata`                    | `diaSource.psfNdata` ? |
| "Weighted mean of the PSF flux forced photometered at the diaSource position on the calibrated image" | `diao.gTOTFluxMean` | `diaObjectForcedSource.psfFlux`? but only for g band sources|
| "Standard deviation of the PSF flux forced photometered at the diaSource position on the calibrated image" | `diao.gTOTFluxSigma` | *(Not in schema?)* |
| Forced DIA Object ID    | `fsodo.diaObjectId`                    | `diaObjectForcedSource.forcedSourceOnDiaObjectId` |
| CCD Visit ID            | `fsodo.ccdVisitId`                     | *(Not in schema?)*    |
| Band / Filter           | `fsodo.band as Filter`                 | `diaObjectForcedSource.band`        |
| PSF Flux                | `fsodo.psfFlux`                        | `diaObjectForcedSource.psfFlux`     |
| PSF Flux Error          | `fsodo.psfFluxErr`                     | `diaObjectForcedSource.psfFluxErr`  |
| Epoch (Midpoint MJD)    | `cv.expMidptMJD as Epoch`              | `diaObjectForcedSource.midpointMjdTai` |
| AB Magnitude            | `scisql_nanojanskyToAbMag(fsodo.psfFlux)` | From `psfFlux` |



In [ ]:
query = "SELECT "\
        "diao.diaObjectId as Object_ID, "\
        "diao.ra as RA, diao.decl as DEC, "\
        "diao.gPSFluxNdata, "\
        "diao.gTOTFluxMean, diao.gTOTFluxSigma, "\
        "fsodo.diaObjectId, "\
        "fsodo.ccdVisitId, "\
        "fsodo.band as Filter, "\
        "fsodo.psfFlux, fsodo.psfFluxErr, "\
        "cv.expMidptMJD as Epoch, "\
        "scisql_nanojanskyToAbMag(fsodo.psfFlux) as Mag "\
        \
        "FROM dp02_dc2_catalogs.DiaObject as diao "\
        "JOIN dp02_dc2_catalogs.ForcedSourceOnDiaObject as fsodo "\
        "ON fsodo.diaObjectId = diao.diaObjectId "\
        "JOIN dp02_dc2_catalogs.CcdVisit as cv ON cv.ccdVisitId = fsodo.ccdVisitId "\
        \
        "WHERE diao.gTOTFluxSigma/diao.gTOTFluxMean > 0.15 "\
        "AND diao.gPSFluxNdata > 60 "


#results = service.search(query)
#sources = results.to_table().to_pandas()

In [22]:
rubin_data.query("diaSource.psfNdata > 60").compute()

2025-03-26 18:29:40,626 - distributed.worker - ERROR - Compute Failed
Key:       ('read_pixel-_to_string_dtype-nestedframe-lambda-e1c870ae1d32bcecc6bc4ce9541a20d8', 98)
State:     executing
Task:  <Task ('read_pixel-_to_string_dtype-nestedframe-lambda-e1c870ae1d32bcecc6bc4ce9541a20d8', 98) _execute_subgraph(...)>
Exception: 'AttributeError("\'Series\' object has no attribute \'psfNdata\'")'
Traceback: '  File "/sdf/group/rubin/sw/conda/envs/lsst-scipipe-10.0.0/lib/python3.12/site-packages/dask/dataframe/core.py", line 97, in apply_and_enforce\n    df = func(*args, **kwargs)\n         ^^^^^^^^^^^^^^^^^^^^^\n  File "/sdf/home/o/olynn/.local/lib/python3.12/site-packages/nested_dask/core.py", line 541, in <lambda>\n    return self.map_partitions(lambda x: npd.NestedFrame(x).query(expr), meta=self._meta)\n                                         ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^\n  File "/sdf/home/o/olynn/.local/lib/python3.12/site-packages/nested_pandas/nestedframe/core.py", line 547, in quer

AttributeError: 'Series' object has no attribute 'psfNdata'

In [ ]:
# Uncomment the following line to view the results table
#sources

The total number of measurements and the number of unique `DiaObjects` found are:

In [ ]:
print(len(sources), len(np.unique(sources['Object_ID'])))

In [ ]:
# Uncomment the following line to save the pandas DataFrame to a pickle file
#sources.to_pickle("my_sources.pkl")

In [ ]:
# Uncomment the following line to read the pandas DataFrame from the pickle file
#candidate_sources = pd.read_pickle("my_sources.pkl")

### 2.2 Query Truth Tables to check if the retrieved candidates are variable stars

For each one of the variable candidates retrieved in Sec. 2.1, query the `Object` table for the position (RA, DEC) of the Object of interest to find the ObjectID, specifying a very small area to search (0.001 degrees) to retrieve only the Object of interest.

Then the ObjectID can be used to pick out the characteristics of interest from the Truth Tables (by use of a WHERE statement in the query and joining different tables).
The `MatchesTruth` table provides identifying information to pick out objects that have matches to truth objects, and extract the simulated truth values from the `TruthSummary` table. This requires joining all three tables (`Object`, `MatchesTruth`, `TruthSummary`).

In [ ]:
sources_obj = pd.DataFrame()   # new DataFrame

for objID in sources['Object_ID'].unique():   # for all Objects retrieved in Sec. 2.1
    print(objID)
    source_sel = sources[sources['Object_ID']==objID].iloc[0]   # select the first one
    RA_sel = source_sel['RA']
    DEC_sel = source_sel['DEC']
    
    query = "SELECT TOP 100 \n"\
            "obj.coord_ra, obj.coord_dec, obj.objectId, \n"\
            "ts.truth_type, ts.is_variable, ts.is_pointsource, \n"\
            "obj.detect_isPrimary, \n"\
            "obj.refBand, obj.refExtendedness, \n"\
            "mt.id_truth_type, mt.match_objectId \n"\
            \
            "FROM dp02_dc2_catalogs.MatchesTruth AS mt \n"\
            "JOIN dp02_dc2_catalogs.TruthSummary AS ts ON mt.id_truth_type = ts.id_truth_type \n"\
            "JOIN dp02_dc2_catalogs.Object AS obj ON mt.match_objectId = obj.objectId \n"\
            \
            "WHERE CONTAINS(POINT('ICRS', obj.coord_ra, obj.coord_dec), \n"\
            f"CIRCLE('ICRS', {RA_sel}, {DEC_sel}, 0.001)) = 1 \n"\
            "AND detect_isPrimary = 1"

    results = service.search(query)
                    
    new_objs = results.to_table().to_pandas()
    sources_obj = pd.concat([sources_obj, new_objs], ignore_index=True)   # new resulting DataFrame

In [ ]:
# View the results table
sources_obj

All the 14 candidates are classified as variable stars and point sources.

In [ ]:
# Uncomment the following line to save the pandas DataFrame to a pickle file
#sources_obj.to_pickle("my_sources_obj.pkl")

In [ ]:
# Uncomment the following line to read the pandas DataFrame from the pickle file
#truth_sources = pd.read_pickle("my_sources_obj.pkl")

In [ ]:
# We define a catalog using the 14 sources retrived in Sec. 2.1 including all the necessary data for our SF analysis
catalog_df = sources[['Object_ID', 'Epoch', 'Mag', 'Filter']]
print(catalog_df)

## 3. Structure Functions Analysis

### 3.1 Define the functions implemented for the SFs analysis

The implemented functions are:

- **data_summary:** Gives a summary of the data used for the analysis
- **calcSF:** Calculates the structure function
- **SF_analysis:** Performs structure function analysis checking the data, calculating the SF (using calcSF) and plotting the results

In [ ]:
### Data summary

def data_summary(df):
    objects = df['Object_ID'].unique()   # list of unique object IDs
    print(f"Objects number = {objects.size}")

    print('')
    print(f"Objects list:", end=" ")
    for obj in objects:
        print(f"\n{obj}:", end=" ")
        obj_data_all = df[df['Object_ID'] == obj]   # select only data related to the object
        for filt in filters:
            obj_data_filter = obj_data_all[obj_data_all['Filter'] == filt]   # select only data related to the filter
            if not obj_data_filter.empty:
                print(f"{filt} ({len(obj_data_filter):3})", end=" ")
    print('')

In [ ]:
### Calculate structure function

@njit   # accelerate the calculations
def calcSF(taumin,taumax,tclip,fclip,nstep):
    dtau = (np.log10(taumax) - np.log10(taumin))/nstep

    tau1 = np.zeros(nstep)
    tau2 = np.zeros(nstep)
    N = np.zeros(nstep)
    SF = np.zeros(nstep)

    for k in range(nstep):
        tau1[k] = 10**(np.log10(taumin) + k*dtau)
        tau2[k] = 10**(np.log10(tau1[k]) + dtau)
        for i in range(tclip.size):
            # Sliced
            dist = tclip[i:] - tclip[i]
            sel = (dist >= tau1[k]) & (dist < tau2[k])
            N[k] += tclip[i:][sel].size
            if tclip[i:][sel].size > 0:
                SF[k] += np.sum((fclip[i:][sel] - fclip[i])**2)
        SF[k] = np.sqrt(SF[k] / N[k]) if N[k]>0 else np.nan
    return(tau1,tau2,SF,N)
            

In [ ]:
### Structure function analysis (calls calcSF function)    

def SF_analysis(df,nstep=200,Nmin=10):
    objects = df['Object_ID'].unique()   # list of unique object IDs
    
    for obj in objects:
        fig_lc, ax_lc = plt.subplots(figsize=(10,5))   # size of figures
        ax_lc.set_title(f'{obj}')
        fig_sf, ax_sf = plt.subplots(figsize=(10,5))
        ax_sf.set_title(f'{obj}')

        obj_data_all = df[df['Object_ID'] == obj]   # select only data related to the object

        for filt in filters:
            obj_data_filter = obj_data_all[obj_data_all['Filter'] == filt]   # select only data related to the filter
            if obj_data_filter.empty: continue
            obj_data_filter = obj_data_filter.drop_duplicates(subset='Epoch', keep='first')   # drop rows with duplicated time (leaves the first one)
            obj_data_filter.sort_values(by=['Epoch'], inplace=True)
            obj_data = obj_data_filter[obj_data_filter['Mag'] > 0]   # select only rows with flux > 0
            count = obj_data['Mag'].count()
            count_init = count
            count_old = count+1   # force entering clipping loop 
            excluded_points = [[],[]]
            while(count_old-count > 0):   # iteratively select only data with flux within 5*sigma
                mag_mean = obj_data['Mag'].mean()   # calculate flux average
                mag_std = obj_data['Mag'].std(ddof=1)   # calculate standard deviation (norm with N-1)
                print(f"{obj}, {filt}: count={count}; mean={mag_mean}; std={mag_std}",end=', ')
                obj_data_excluded = obj_data[np.abs(obj_data['Mag']-mag_mean) >= 5*mag_std]
                excluded_points[0] += obj_data_excluded['Epoch'].to_list()
                excluded_points[1] += obj_data_excluded['Mag'].to_list()
                obj_data = obj_data[np.abs(obj_data['Mag']-mag_mean) < 5*mag_std]   # select only data with flux within 5*sigma
                count_old = count
                count = obj_data['Mag'].count()
                print(f"count after clipping: {count}")
            mag_mean = obj_data['Mag'].mean()   # calculate flux average
            mag_std = obj_data['Mag'].std(ddof=1)   # calculate standard deviation (norm with N-1)
            print(f"Init. count={count_init:5d}; clip count={count:5d}; mean={mag_mean:.6f}; std={mag_std:.6f}")

            # Uncomment to print the excluded points
            #if len(excluded_points[0]) > 0:
            #    print(f"Excluded points: {np.array(excluded_points[0]),excluded_points[1]}")

            if obj_data.empty: continue

            tclip = obj_data['Epoch'].to_numpy()
            fclip = obj_data['Mag'].to_numpy()
            fclip = 10**(-0.4*(fclip-np.median(fclip)))   # convert magnitudes to fluxes

            dtmin = np.min(np.diff(tclip))
            dtmax = tclip[-1]-tclip[0] 

            print(f"Minimum time lag is {dtmin:.6f}, maximum time lag is {dtmax:.6f}")

            taumin = 2.*dtmin
            taumax = 0.5*dtmax

            print(f"Minimum considered time lag is {taumin:.6f}, maximum considered time lag is {taumax:.6f}")

            tau1,tau2,SF,N = calcSF(taumin,taumax,tclip,fclip,nstep)

            print('')
            
            # Uncomment to save the output
            #np.savetxt('out_'+obj+'_'+filt+'.txt', np.c_[tau1, tau2, SF, N], delimiter='\t',header="tau1, tau2, SF, N")

            ax_lc.plot(obj_data['Epoch'], obj_data['Mag'], label=filt, color=colors[filt], marker=symbols[filt], linewidth=0)
            # Uncomment to plot the excluded points
            #if len(excluded_points[0]) > 0:
            #    ax_lc.plot(np.array(excluded_points[0]),excluded_points[1],'+',label=filt,color=colors[filt])
            ax_sf.loglog(tau1[N>Nmin], SF[N>Nmin], label=filt, color=colors[filt], marker=symbols[filt], linewidth=0)

        ax_lc.set_xlabel("MJD")
        ax_lc.set_ylabel("Mag")
        ax_lc.invert_yaxis()
        ax_lc.grid(True)
        ax_lc.legend()

        ax_sf.set_xlabel("tau1")
        ax_sf.set_ylabel("SF(tau1,tau2)")
        ax_sf.grid(True)
        ax_sf.legend()

        print('-'*99)

    plt.show()


### 3.2 Application to the variable stars selected from the DP0.2 Catalogs

Apply the SF analysis to the data retrieved in Section 2.

In [ ]:
## How the data looks like (ObjectsID, Filters and Number of points)
data_summary(catalog_df)

Calculate the SF for each Object and Filter, and plot both the input lightcurve and the output SF.

In [ ]:
SF_analysis(catalog_df,nstep=50,Nmin=2)

### 3.3 Application to two examples of variable YSOs

Apply the SF analysis to Mon-63 (in NGC2264) and to V2492_Cyg (EXor) reading the datafiles provided in the current folder.

In [ ]:
## Read data file
datafile = "NGC2264_Mon-63_lc.txt"
Mon63_df = pd.read_csv(datafile, sep='\t', header=0)   # import data as pandas dataframe
#print(Mon63_df)

In [ ]:
## Data summary
data_summary(Mon63_df)

Calculate the SF for each Filter, and plot both the input lightcurve and the output SF.

In [ ]:
SF_analysis(Mon63_df,Nmin=5)

In [ ]:
## Read data file
datafile = "V2492_Cyg_lc.txt"
V2492Cyg_df = pd.read_csv(datafile, sep='\t', header=0)   # import data as pandas dataframe (EXor)
#print(V2492Cyg_df)

In [ ]:
## Data summary
data_summary(V2492Cyg_df)

Calculate the SF for each Filter, and plot both the input lightcurve and the output SF.

In [ ]:
SF_analysis(V2492Cyg_df)

## 4. Continue exploring the data

There are many further explorations one could try as extensions of this notebook. Some examples:

1. Experiment with ways to select the variable objects in the Catalogs.
2. Try to apply the SF analysis to other variable objects.
3. Explore how the calculated SF changes when varying the nstep input parameter.
4. Extract the characteristic statistical values for the SFs calculated.
